# Test du modèle (Resnet18)

Dans ce notebook, nous allons tester le modèle afin de nous familiariser avec son utilisation

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import shutil

from tensorflow import keras

# Création du modèle

In [ ]:
import classification_models
from classification_models.tfkeras import Classifiers

# Chargement du modèle ResNEt18
ResNet18, preprocess_input = Classifiers.get('resnet18')

# Instanciation du modèle pré-entrainé ResNet18
base_model = ResNet18(input_shape=(224,224,3), weights='imagenet', include_top=False,pooling=False)

# Désactivation des couches pour l'entrainement
for layer in base_model.layers:
    layer.trainable = False
    
# Ajout de l'applatissement des sorties et de la couche dense avec 2 neurones"
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(units=2, activation='softmax')(x)

model = tf.keras.Model(inputs=[base_model.input], outputs=[output])

On charge ensuite les poids sauvegardés lors de l'entrainement :

In [ ]:
model.load_weights("meilleur_modele.hdf5")

### Création de l'interface d'acquisition

J'ai créé une classe Image afin d'optimiser le pipeline avec OpenCV :

In [ ]:
import traitlets
import threading
import atexit
import numpy as np


class Camera(traitlets.HasTraits):
    type_camera = traitlets.Unicode("CSI")
    capture_device = traitlets.Integer(default_value=0)
    capture_width = traitlets.Integer(default_value=1280)
    capture_height = traitlets.Integer(default_value=720)
    display_width = traitlets.Integer(default_value=640)
    display_height = traitlets.Integer(default_value=480)
    fps = traitlets.Integer(default_value=30)
    flip = traitlets.Integer(default_value=0)
    image = traitlets.Any()
    video_on = traitlets.Bool(default_value=False)
    
    def __init__(self,*args,**kwargs):
        super(Camera, self).__init__(*args, **kwargs)
        self._running = False
        self.image = np.zeros((self.display_height, self.display_width, 3), dtype=np.uint8)
        
        if self.type_camera.find("CSI")>=0:
            self.cap = cv2.VideoCapture(self._gstreamer_pipeline_CSI(),cv2.CAP_GSTREAMER)
        else:
            self.cap = cv2.VideoCapture(self._gstreamer_pipeline_USB(),cv2.CAP_GSTREAMER)

        if self.cap.isOpened():
            print("Caméra initialisée")
        else:
            print("Erreur d'ouverture du flux vidéo")
        atexit.register(self.cap.release)
    
    # Lecture d'une frame
    def capture_image(self):
        re, image = self.cap.read()
        if re:
            image_resized = cv2.resize(image,(int(self.display_width),int(self.display_height)))
            return image_resized
        else:
            return self.image
    
    # ON/OFF de la capture vidéo
    def capture_video(self,run=False):
        if run is True:
            self.video_on = True
        else:
            self.video_on = False
    
    # Lecture d'un flux vidéo
    def _capture_video(self):
        while True:
            if not self._running:
                break
            self.image = self.capture_image()

            
    # Détachement de la caméra
    def release(self):
        self.cap.release()

    # Définition du pipeline pour la caméra CSI
    def _gstreamer_pipeline_CSI(self):
        return("nvarguscamerasrc sensor-id=%d ! "
                "video/x-raw(memory:NVMM),"
                "width=(int)%d,height=(int)%d,"
                "format=(string)NV12, framerate=(fraction)%d/1 ! "
                "nvvidconv flip-method=%d ! "
                "video/x-raw,"
                "width=(int)%d,height=(int)%d,"
                "format=(string)BGRx ! videoconvert ! "
                "video/x-raw, format=(string)BGR ! "
                "appsink drop=true"
        %(self.capture_device,self.capture_width,self.capture_height,self.fps,self.flip, self.display_width,self.display_height))

    # Définition du pipeline pour la USB
    def _gstreamer_pipeline_USB(self):
        return("v4l2src device=/dev/video%d ! "
               "video/x-raw, width=(int)%d, height=(int)%d, framerate=(fraction)%d/1 ! "
               "videoflip method=%d ! "
               "videoconvert ! "
               "video/x-raw, format=(string)BGR ! appsink drop=true"
        %(self.capture_device,self.capture_width,self.capture_height,self.fps,self.flip))
    
    # Surveillance de la variable "video_on"
    @traitlets.observe('video_on')
    def _on_running(self, change):
        if change['new'] and not change['old']:
            # not running -> running
            self._running = True
            self.thread = threading.Thread(target=self._capture_video)
            self.thread.start()
        elif change['old'] and not change['new']:
            # running -> not running
            self._running = False
            self.thread.join()

Fonction pour initialiser la caméra :

In [ ]:
def InitCamera():
    camera = Camera(type_camera="CSI",capture_device=0,
                capture_width=224,capture_height=224,
                display_width=224,display_height=224,
                fps=25,flip=0)
    return camera

Création de l'interface :

In [ ]:
import ipywidgets
import traitlets
from IPython.display import display

try :
    camera.capture_video(run=False)
    camera.release()
    del camera
except NameError:
    pass

# Initialise la caméra
camera = InitCamera()

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets

from jetbot import bgr8_to_jpeg

# Lance la caméra
camera.capture_video(run=True)

# Création de l'interface
image = widgets.Image(format='jpeg', width=224, height=224)

lien_camera = traitlets.dlink((camera, 'image'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

### Utilisation du modèle

On commence par récupérer l'image :

In [ ]:
image_camera = camera.image
image_camera.shape

On envoit ensuite l'image dans le réseau :

In [ ]:
image_camera = camera.image

image_camera = tf.expand_dims(image_camera,0)        # (224,224,3) => (1,224,224,3)
probas_labels = model(image_camera)                  # Appel du modèle
probas_labels.shape

Regardons à quoi ressemble la variable récupérée :

In [ ]:
probas_labels

### Fin de la session

Avant de quitter, il faut fermer la caméra :

In [ ]:
try :
    camera.capture_video(run=False)
    camera.release()
    del camera
except NameError:
    pass